created on: Thu Jan 16 13:35:03 2020
<br>
Group 7
<br>
@authors: V.B., C.L.

This notebook aims to analyse scores resulting from our models, after all the images went through the pipeline.
Depending on the chosen model, you would have to change the type_ext and type_int lists.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt


In [19]:
path_project = './../'
path_pred = path_project + 'Results/'
path_real = path_project + 'ImagesStats/'

In [4]:
df_pred = pd.read_csv(path_pred + 'pred_SEATGURU_4.csv', sep=';')
df_real = pd.read_csv(path_real + 'g7_SEATGURU_annotate.csv', sep=';')


In [21]:
man = ['Airbus', 'Boeing']

# Types for which exterior model was trained
types_ext = ['A320','A321','A330','A340','A350','A380','737','747','757','777', '787']


# Types for which exterior model was trained
int_airb = ['A320','A321','A330','A340','A350']
int_boeing = ['737','747','757','777', '787']
types_int = int_airb + int_boeing


In [6]:
df_real.rename(columns={"aircraft_manufacturer": "manufacturer", "aircraft_type" : "type", "name": "img"}, inplace = True)

In [7]:
df = pd.merge(df_real, df_pred, on = "img")

In [8]:
df = df.drop(columns = ['format', 'height', 'width', 'height_to_width', 'ncol'])

In [9]:
df

,img,manufacturer_x,type_x,view_x,view_y,view_proba,manufacturer_y,manufacturer_proba,type_y,type_proba
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,Boeing,777,Meal,Meal,0.999996,NaN,NaN,NaN,NaN
1,KLM_Airbus_A330-300_1.jpg,Airbus,A330,Int,Int,0.999998,Airbus,0.998750,A330,0.998699
2,American_Airlines_Boeing_767-300_3.jpg,Boeing,767,Int,Int,1.000000,Boeing,0.999102,777,0.998976
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,Boeing,767,Meal,Meal,0.999946,NaN,NaN,NaN,NaN
4,United_Airlines_Q400_A_2.jpg,Other,NaN,Ext,Ext,0.999990,Boeing,NaN,757,0.446160
...,...,...,...,...,...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,Airbus,A330,Int,Int,0.999976,Airbus,0.999686,A320,0.604245
2552,Qatar_Airways_Airbus_A321_2.jpg,Airbus,A321,Int,Int,0.999851,Airbus,0.997766,A321,0.380941
2553,Frontier_Airlines_Airbus_A320_4.jpg,Airbus,A320,Ext_Int,Ext_Int,0.999627,NaN,NaN,NaN,NaN
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,Airbus,A320,Int,Int,0.998991,Airbus,0.998399,A320,0.998727


In [10]:
df_view = df[df.apply(lambda x : x['view_x'] == x['view_y'] , axis = 1)]
print('Score for the viewpoint : ')
print(len(df_view)/len(df[df['view_x'] != 'Others'])) 
print(len(df_view)/len(df))

Score for the viewpoint : 
0.9899879855826993
0.9671361502347418


In [11]:
df_ext_pred_OK = df[(df['view_x'] == 'Ext') & (df['view_y'] == 'Ext')]

df_ext_ab = df_ext_pred_OK[df_ext_pred_OK['manufacturer_x'].isin(man)]
df_ext_man_OK = df_ext_ab[df_ext_ab['manufacturer_x'] == df_ext_ab['manufacturer_y']]
print('Score for manufacturer detection for planes detected as "Ext" : ')
print(len(df_ext_man_OK)/len(df_ext_ab))
print('With others manufacturers : ')
print(len(df_ext_man_OK)/len(df_ext_pred_OK))


Score for manufacturer detection for planes detected as "Ext" : 
0.5842105263157895
With others manufacturers : 
0.4065934065934066


In [12]:
df_type_ab = df_ext_pred_OK[df_ext_pred_OK['type_x'].isin(types_ext)]
df_ext_type_OK = df_ext_pred_OK[df_ext_pred_OK['type_x'] == df_ext_pred_OK['type_y']]
print('Score for airplane detection for planes detected as "Ext" (for aircraft included in the training): ')
print(len(df_ext_type_OK)/len(df_type_ab))

Score for airplane detection for planes detected as "Ext" (for aircraft included in the training): 
0.14124293785310735


In [13]:
df_int_pred_OK = df[(df['view_x'] == 'Int') & (df['view_y'] == 'Int')]

df_int_ab = df_int_pred_OK[df_int_pred_OK['manufacturer_x'].isin(man)]
df_int_man_OK = df_int_ab[df_int_ab['manufacturer_x'] == df_int_ab['manufacturer_y']]
print('Score for manufacturer detection for planes detected as "Int" (with only Airbus and Boeing): ')
print(len(df_int_man_OK)/len(df_int_ab))
print('Score for manufacturer detection for planes detected as "Int" (with all manufacturers): ')
print(len(df_int_man_OK)/len(df_int_pred_OK))


Score for manufacturer detection for planes detected as "Int" (with only Airbus and Boeing): 
0.8822792736380713
Score for manufacturer detection for planes detected as "Int" (with all manufacturers): 
0.7575268817204301


In [93]:
df_type_ab = df_int_pred_OK[df_int_pred_OK['type_x'].isin(types_int)]
df_int_type_OK = df_int_pred_OK[df_int_pred_OK['type_x'] == df_int_pred_OK['type_y']]
print('Score for airplane detection for planes detected as "Int" : ')
print(len(df_int_type_OK)/len(df_type_ab))

Score for airplane detection for planes detected as "Int" : 
0.6463326071169209


In [23]:
df_int_airbus = df_int_pred_OK[df_int_pred_OK['type_x'].isin(int_airb)]
df_int_airbus_OK = df_int_airbus[df_int_airbus['type_x']==df_int_airbus['type_y']]
print('Score for airplane detection for planes detected as Airbus interior (for types included in train) : ')
len(df_int_airbus_OK)/len(df_int_airbus)



Score for airplane detection for planes detected as Airbus interior (for types included in train) : 


0.6516129032258065

In [24]:
df_int_boeing = df_int_pred_OK[df_int_pred_OK['type_x'].isin(int_boeing)]
df_int_boeing_OK = df_int_boeing[df_int_boeing['type_x']==df_int_boeing['type_y']]
print('Score for airplane detection for planes detected as Airbus interior (for types included in train) : ')
len(df_int_boeing_OK)/len(df_int_boeing)


Score for airplane detection for planes detected as Airbus interior (for types included in train) : 


0.6427566807313643

In [14]:
# Dataframes with predictions errors

df['pred_OK'] = df.apply(lambda x :  x['view_x'] == x['view_y'], axis = 1)
df_view = df[['img', 'view_x', 'view_y', 'view_proba', 'pred_OK']]

df_view_OK = df_view[df_view['pred_OK']== True]
df_view_OK

,img,view_x,view_y,view_proba,pred_OK
0,Cathay_Pacific_Airways_Boeing_777-300ER_C_0.jpg,Meal,Meal,0.999996,True
1,KLM_Airbus_A330-300_1.jpg,Int,Int,0.999998,True
2,American_Airlines_Boeing_767-300_3.jpg,Int,Int,1.000000,True
3,Air_Canada_Boeing_767-300ER_v2_3.jpg,Meal,Meal,0.999946,True
4,United_Airlines_Q400_A_2.jpg,Ext,Ext,0.999990,True
...,...,...,...,...,...
2551,Air_Canada_Airbus_A330_C_0.jpg,Int,Int,0.999976,True
2552,Qatar_Airways_Airbus_A321_2.jpg,Int,Int,0.999851,True
2553,Frontier_Airlines_Airbus_A320_4.jpg,Ext_Int,Ext_Int,0.999627,True
2554,Spirit_Airlines_Airbus_A320_V2_3.jpg,Int,Int,0.998991,True


In [15]:
df_view_err_no_others = df_view[(df_view['pred_OK']==False) &(df_view['view_x']!='Others')]
df_view_err_no_others

,img,view_x,view_y,view_proba,pred_OK
180,KLM_Boeing_777-200ER_2.jpg,Int,Meal,0.646653,False
298,Austrian_Airlines_Airbus_A320_4.jpg,Ext_Int,Ext,0.569161,False
356,United_Airlines_Boeing_737-900_E_4.jpg,Ext,Ext_Int,0.963786,False
494,Turkish_Airlines_Airbus_A330-300_2.jpg,Int,Meal,0.999670,False
617,Delta_Airlines_Boeing_767-300ER_C_0.jpg,Int,Meal,0.990855,False
632,Hainan_Airlines_Airbus_A330-300_0.jpg,Ext_Int,Ext,0.998867,False
708,Cathay_Pacific_Airways_Boeing_777-300_B_4.jpg,Meal,Int,0.849697,False
845,British_Airways_Boeing_787-9_1.jpg,Ext_Int,Int,0.661129,False
932,Hawaiian_Airlines_ATR_42-500_0.jpg,Ext_Int,Ext,0.999878,False
978,TAM_Airlines_Boeing_767-300ER_4.jpg,Meal,Int,0.759586,False


In [16]:
df_view_err_others = df_view[(df_view['pred_OK']==False) &(df_view['view_x']=='Others')]
df_view_err_others

,img,view_x,view_y,view_proba,pred_OK
9,Iberia_Airbus_A330-200_0.jpg,Others,Int,0.993340,False
11,Oman_Air_Boeing_737-900_3.jpg,Others,Meal,0.988062,False
64,United_Airlines_Bombardier_Q300_2.jpg,Others,Meal,0.980661,False
90,Lufthansa_Boeing_747_8_V3_0.jpg,Others,Int,0.997195,False
96,Malaysia_Airlines_Airbus_A380_new_4.jpg,Others,Ext,0.534779,False
123,Avianca_Airbus_A320-200_1.jpg,Others,Int,0.844808,False
155,Copa_Airlines_Boeing_737-700_4.jpg,Others,Int,0.998061,False
171,Qantas_Airways_Airbus_A330-300_2.jpg,Others,Int,0.546983,False
203,Nok_Air_Boeing_737-800_3.jpg,Others,Int,1.000000,False
243,Royal_Jordanian_Boeing_787-8_4.jpg,Others,Int,0.883760,False
